In [1]:
import tensorflow as tf 
from tensorflow.keras.layers import *


2022-12-31 03:50:18.778221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 03:50:19.344345: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/cuda-11.7/lib64${LD_LIBRARY_PATH:+:}
2022-12-31 03:50:19.344527: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object fi

In [88]:
input=tf.keras.Input(shape=(64,288,288,3))
x=Conv3D(filters=64,kernel_size=(4,8,8),strides=(4,8,8),activation='relu')(input)

x=Conv3D(filters=256,kernel_size=(2,4,4),strides=(2,4,4),activation='relu')(x)
x=Conv3D(filters=256,kernel_size=(2,2,2),activation='relu')(x)
x=MaxPool3D()(x)
# x=Conv3D(filters=256,kernel_size=(4,4,4),strides=(1,1,1),activation='relu')(x)
# x=MaxPool3D()(x)




x=Reshape(target_shape=(-1,256))(x)
model=tf.keras.Model(input,x)




In [89]:
model.summary()

Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_49 (InputLayer)       [(None, 64, 288, 288, 3)  0         
                             ]                                   
                                                                 
 conv3d_101 (Conv3D)         (None, 16, 36, 36, 64)    49216     
                                                                 
 conv3d_102 (Conv3D)         (None, 8, 9, 9, 256)      524544    
                                                                 
 conv3d_103 (Conv3D)         (None, 7, 8, 8, 256)      524544    
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 3, 4, 4, 256)     0         
 3D)                                                             
                                                                 
 reshape_38 (Reshape)        (None, 48, 256)           0  

In [17]:
288/8


36.0

In [102]:
class TubeletEmbedding(tf.keras.layers.Layer):

    def __init__(self,d_modal,dropout=0.5,**kwargs):
        super(TubeletEmbedding,self).__init__(**kwargs)
        self.d_modal=d_modal
        self.conv1=Conv3D(filters=self.d_modal/4,kernel_size=(4,8,8),strides=(4,8,8),activation='relu')
        self.conv2=Conv3D(filters=self.d_modal,kernel_size=(2,4,4),strides=(2,4,4),activation='relu')
        self.conv3=Conv3D(filters=self.d_modal,kernel_size=(2,2,2),activation='relu')
        self.maxpool3d=MaxPool3D()
        self.reshape=tf.keras.layers.Reshape(target_shape=(-1,self.d_modal))
        self.dropout=dropout

    def call(self,videos):


        x=self.conv1(videos)
        x=self.conv2(x)
        x=tf.keras.layers.Dropout(self.dropout)(x)
        x=self.conv3(x)
        x=tf.keras.layers.Dropout(self.dropout)(x)
        x=self.maxpool3d(x)
        x=self.reshape(x)
        return x

        

In [103]:
sample_input=tf.random.normal(shape=(1,64,288,288,3))

In [107]:
x=TubeletEmbedding(256,0.5)(sample_input)

In [108]:
x

<tf.Tensor: shape=(1, 48, 256), dtype=float32, numpy=
array([[[0.15994602, 0.08972547, 0.05428707, ..., 0.27504224,
         0.16010995, 0.37122452],
        [0.13152602, 0.        , 0.08426345, ..., 0.1963974 ,
         0.09561629, 0.172574  ],
        [0.05773117, 0.04975312, 0.03860175, ..., 0.2699294 ,
         0.1258084 , 0.1900611 ],
        ...,
        [0.18437806, 0.04404083, 0.06679596, ..., 0.2438691 ,
         0.0907255 , 0.25153333],
        [0.16808245, 0.02926301, 0.08175955, ..., 0.17571384,
         0.18058157, 0.22221226],
        [0.16402996, 0.06398805, 0.14502162, ..., 0.31113693,
         0.11757857, 0.17629236]]], dtype=float32)>

: 

In [ ]:
""